In [1]:
import logging
import warnings

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

warnings.filterwarnings("ignore")

from freamon import Freamon

In [2]:
pipeline, runtimes = Freamon().pipeline_from_py_file('pipelines--mlinspect--amazon-reviews.py', cmd_args=[])

INFO:root:Patching sys.argv with ['eyes']
INFO:root:Executing instrumented user pipeline with mlinspect
INFO:root:Redirecting the pipeline's stdout to pipeline-output.txt
INFO:root:---RUNTIME: Instrumented execution took 11210.924625396729 ms
INFO:root:Identifying training sources
INFO:root:Duplicate row 0 found for operator 3 in polynomial {LineageId(operator_id=1, row_id=20859), LineageId(operator_id=3, row_id=0), LineageId(operator_id=0, row_id=65402), LineageId(operator_id=2, row_id=2)}
INFO:root:Duplicate row 2 found for operator 2 in polynomial {LineageId(operator_id=1, row_id=20859), LineageId(operator_id=3, row_id=0), LineageId(operator_id=0, row_id=65402), LineageId(operator_id=2, row_id=2)}
INFO:root:rows_from_operator.keys() dict_keys([2, 1, 3, 0])
INFO:root:operators_with_duplicates {2, 3}
INFO:root:sources_one_to_one 2
INFO:root:Found dimension table from operator 2 with 10932 records and the following attributes: ['product_id', 'product_parent', 'product_title', 'category

In [3]:
pipeline.dag

In [4]:
def find_by_type(graph, operator_type):
    for node in graph.nodes:
        if node.operator_info.operator == operator_type:
            return node
        
def find_spj(graph, node, ops):
    for parent in pipeline.dag.predecessors(node):
        if parent.operator_info.operator == OperatorType.JOIN:
            ops.add(parent)
            
        find_spj(graph, parent, ops)
        

In [5]:
from mlinspect.inspections._inspection_input import OperatorType

train_data_node = find_by_type(pipeline.dag, OperatorType.TRAIN_DATA)

ops = set()

find_spj(pipeline.dag, train_data_node, ops)

print(ops)

#for node in pipeline.dag.nodes:
#    if node.operator_info.operator == :
#        for parent in pipeline.dag.predecessors(node):
#            print(parent)

{DagNode(node_id=8, code_location=BasicCodeLocation(caller_filename='pipelines--mlinspect--amazon-reviews.py', lineno=40), operator_info=OperatorContext(operator=<OperatorType.JOIN: 'Join'>, function_info=FunctionInfo(module='pandas.core.frame', function_name='merge')), details=DagNodeDetails(description="on 'product_id'", columns=['marketplace', 'customer_id', 'review_id', 'product_id', 'vine', 'third_party', 'review_headline', 'review_body', 'review_date', 'star_rating', 'helpful_votes', 'total_votes', 'product_parent', 'product_title', 'category_id', 'id', 'category']), optional_code_info=OptionalCodeInfo(code_reference=CodeReference(lineno=40, col_offset=40, end_lineno=40, end_col_offset=109), source_code="reviews_with_ratings.merge(products_with_categories, on='product_id')")), DagNode(node_id=5, code_location=BasicCodeLocation(caller_filename='pipelines--mlinspect--amazon-reviews.py', lineno=34), operator_info=OperatorContext(operator=<OperatorType.JOIN: 'Join'>, function_info=Fu

In [105]:
import duckdb
# to start an in-memory database
con = duckdb.connect(database=':memory:')
                     
from freamon.templates._outputs import Output    
for source in pipeline.test_sources:    
    con.register(f'_freamon_source_{source.operator_id}', source.data)    
    

In [106]:
from mlinspect.inspections._inspection_input import OperatorType

x_test = pipeline.dag_node_to_intermediates[find_by_type(pipeline.dag, OperatorType.TEST_DATA)]
y_test = pipeline.dag_node_to_intermediates[find_by_type(pipeline.dag, OperatorType.TEST_LABELS)]
y_pred = pipeline.dag_node_to_intermediates[find_by_type(pipeline.dag, OperatorType.SCORE)]

#con.register(f'_freamon_x_test', x_test)    

import pandas as pd

x_test.rename(columns={'array': 'features'}, inplace=True)
x_test['y_true'] = y_test['array']
x_test['y_pred'] = y_pred['array']

con.register(f'_freamon_x_test', x_test) 

In [107]:
x_test

,features,mlinspect_lineage,y_pred,y_true
0,"[-0.0010078951380543934, -1.1586623990409706, ...","(0,0);(1,46881);(2,0);(3,0);(0,0);(1,46881);(2...",0,0
1,"[0.02370451284938802, 0.7684678523973569, 1.0,...","(0,453);(1,74300);(2,0);(3,0);(0,453);(1,74300...",0,0
2,"[-0.07514511910038163, 0.7684678523973569, 1.0...","(0,2);(1,51654);(2,2);(3,0);(0,2);(1,51654);(2...",1,1
3,"[-0.0010078951380543934, -1.1586623990409706, ...","(0,1202);(1,28309);(2,2);(3,0);(0,1202);(1,283...",0,1
4,"[-0.07514511910038163, 0.7684678523973569, 1.0...","(0,4853);(1,42667);(2,2);(3,0);(0,4853);(1,426...",1,1
...,...,...,...,...
25950,"[-0.025720303125496804, -1.8010391495204132, 1...","(0,63663);(1,61853);(2,3600);(3,0);(0,63663);(...",1,1
25951,"[-0.07514511910038163, -1.1586623990409706, 1....","(0,63703);(1,85708);(2,5605);(3,0);(0,63703);(...",1,1
25952,"[-0.0010078951380543934, 0.7684678523973569, 1...","(0,63747);(1,46227);(2,3240);(3,0);(0,63747);(...",0,0
25953,"[-0.07514511910038163, -1.1586623990409706, 1....","(0,63775);(1,21482);(2,5608);(3,0);(0,63775);(...",1,1


In [108]:
# TODO this might have some issues if the same table is joined twice, because we lose the order in list_distinct
con.execute("""
  CREATE OR REPLACE VIEW _freamon_test_view AS 
  SELECT 
    features, y_true, y_pred,
    CAST(regexp_replace(polynomial[1], '\(\\d+,|\)', '', 'g') AS INT) AS prov_id_source_0,
    CAST(regexp_replace(polynomial[2], '\(\\d+,|\)', '', 'g') AS INT) AS prov_id_source_1,
    CAST(regexp_replace(polynomial[3], '\(\\d+,|\)', '', 'g') AS INT) AS prov_id_source_2,
    CAST(regexp_replace(polynomial[4], '\(\\d+,|\)', '', 'g') AS INT) AS prov_id_source_3,    
  FROM (
    SELECT 
      features, y_true, y_pred,
      list_sort(list_distinct(string_to_array(mlinspect_lineage, ';'))) AS polynomial
    FROM _freamon_x_test
  )
""")

con.fetchall()

[]

In [109]:
con.execute("""
  DESCRIBE _freamon_x_test
""")

con.fetchall()

[('features', 'VARCHAR', 'YES', None, None, None),
 ('mlinspect_lineage', 'VARCHAR', 'YES', None, None, None),
 ('y_pred', 'BIGINT', 'YES', None, None, None),
 ('y_true', 'BIGINT', 'YES', None, None, None)]

In [110]:
con.execute("""
  CREATE OR REPLACE VIEW _freamon_source_0_with_prov_view AS 
  SELECT 
    * EXCLUDE mlinspect_lineage, 
   CAST(string_to_array(trim(mlinspect_lineage, ')('), ',')[2] AS INT) AS prov_id_source_0
  FROM _freamon_source_0""")
con.fetchall()

[]

In [111]:
con.execute("""
  CREATE OR REPLACE VIEW _freamon_source_1_with_prov_view AS 
  SELECT 
    * EXCLUDE mlinspect_lineage, 
   CAST(string_to_array(trim(mlinspect_lineage, ')('), ',')[2] AS INT) AS prov_id_source_1
  FROM _freamon_source_1""")
con.fetchall()

[]

In [112]:
con.execute("""
  CREATE OR REPLACE VIEW _freamon_source_2_with_prov_view AS 
  SELECT 
    * EXCLUDE mlinspect_lineage, 
   CAST(string_to_array(trim(mlinspect_lineage, ')('), ',')[2] AS INT) AS prov_id_source_2
  FROM _freamon_source_2""")
con.fetchall()

[]

In [113]:
con.execute("""
  CREATE OR REPLACE VIEW _freamon_source_3_with_prov_view AS 
  SELECT 
    * EXCLUDE mlinspect_lineage, 
   CAST(string_to_array(trim(mlinspect_lineage, ')('), ',')[2] AS INT) AS prov_id_source_3
  FROM _freamon_source_3""")
con.fetchall()

[]

In [116]:
con.execute("""
SELECT * 
FROM _freamon_test_view ftv
JOIN _freamon_source_0_with_prov_view fs0 ON fs0.prov_id_source_0 = ftv.prov_id_source_0
JOIN _freamon_source_1_with_prov_view fs1 ON fs1.prov_id_source_1 = ftv.prov_id_source_1
JOIN _freamon_source_2_with_prov_view fs2 ON fs2.prov_id_source_2 = ftv.prov_id_source_2
JOIN _freamon_source_3_with_prov_view fs3 ON fs3.prov_id_source_3 = ftv.prov_id_source_3

""")
#con.fetchall()
rectangle = con.df()
rectangle

,features,y_true,y_pred,prov_id_source_0,prov_id_source_1,prov_id_source_2,prov_id_source_3,marketplace,customer_id,review_id,...,total_votes,prov_id_source_1_2,product_id_2,product_parent,product_title,category_id,prov_id_source_2_2,id,category,prov_id_source_3_2
0,[-1.00789514e-03 -1.15866240e+00 1.00000000e+...,0,0,0,46881,0,0,US,21269168,RSH1OZ87OYK92,...,3,46881,B013PURRZW,603406193,Madden NFL 16 - Xbox One Digital Code,0,0,0,Digital_Video_Games,0
1,[-0.07514512 0.76846785 1. ... 0. ...,1,1,2,51654,2,0,US,45765011,R3YOOS71KM5M9,...,0,51654,B00DNHLFQA,951665344,Command & Conquer The Ultimate Collection [Ins...,0,2,0,Digital_Video_Games,0
2,[-0.07514512 0.76846785 1. ... 0. ...,1,1,3,63888,3,0,US,113118,R3R14UATT3OUFU,...,0,63888,B004RMK5QG,395682204,Playstation Plus Subscription,0,3,0,Digital_Video_Games,0
3,[-0.07514512 0.76846785 1. ... 0. ...,1,1,5,47518,5,0,US,22151364,R3CFKLIZ0I2KOB,...,0,47518,B00IMIL498,621922192,Double Dragon: Neon [Online Game Code],0,5,0,Digital_Video_Games,0
4,[-0.07514512 0.1260911 1. ... 0. ...,1,1,6,51214,6,0,US,38426028,R1LRYU1V0T3O38,...,0,51214,B00S00IJH8,215163395,Sims 4,0,6,0,Digital_Video_Games,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25950,[-0.07514512 0.76846785 1. ... 0. ...,1,1,63832,28540,2704,0,US,41379142,R9IPEMD190KBT,...,0,28540,B003BVIXWC,356854136,Prince of Persia: The Forgotten Sands [Download],0,2704,0,Digital_Video_Games,0
25951,[-0.07514512 0.76846785 1. ... 0.03...,1,1,63833,46439,1144,0,US,19286168,RIOIRX9NH6WV4,...,0,46439,B00I0A363A,628527631,Path of Exile,0,1144,0,Digital_Video_Games,0
25952,[ 0.19669137 -1.80103915 1. ... 0. ...,0,0,63834,90386,1371,0,US,4790346,R1EWYSYERLYKM4,...,11,90386,B00GMTKMHY,686779980,The Last of Us: Left Behind DLC - PS3 [Digital...,0,1371,0,Digital_Video_Games,0
25953,[-0.07514512 0.76846785 1. ... 0. ...,1,1,63835,57674,4896,0,US,5223154,R3DWNBLDOWKYRL,...,0,57674,B00GXM48HA,673376391,Xbox $50 Gift Card (Battlefield 4 Premium)- Xb...,0,4896,0,Digital_Video_Games,0


In [119]:
for index, source in enumerate(pipeline.test_sources):
    print(index, list(source.data.columns))

0 ['product_id', 'product_parent', 'product_title', 'category_id', 'mlinspect_lineage']
1 ['id', 'category', 'mlinspect_lineage']
2 ['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'mlinspect_lineage']
3 ['marketplace', 'customer_id', 'review_id', 'product_id', 'vine', 'third_party', 'review_headline', 'review_body', 'review_date', 'mlinspect_lineage']


In [122]:
con.execute("""DESCRIBE _freamon_source_1_with_prov_view""").df()

,column_name,column_type,null,key,default,extra
0,review_id,VARCHAR,YES,NaN,NaN,NaN
1,star_rating,BIGINT,YES,NaN,NaN,NaN
2,helpful_votes,BIGINT,YES,NaN,NaN,NaN
3,total_votes,BIGINT,YES,NaN,NaN,NaN
4,prov_id_source_1,INTEGER,YES,NaN,NaN,NaN


In [122]:
con.execute("""DESCRIBE _freamon_source_1_with_prov_view""").df()

,column_name,column_type,null,key,default,extra
0,review_id,VARCHAR,YES,NaN,NaN,NaN
1,star_rating,BIGINT,YES,NaN,NaN,NaN
2,helpful_votes,BIGINT,YES,NaN,NaN,NaN
3,total_votes,BIGINT,YES,NaN,NaN,NaN
4,prov_id_source_1,INTEGER,YES,NaN,NaN,NaN


In [123]:
con.execute("""DESCRIBE _freamon_source_2_with_prov_view""").df()

,column_name,column_type,null,key,default,extra
0,product_id,VARCHAR,YES,NaN,NaN,NaN
1,product_parent,BIGINT,YES,NaN,NaN,NaN
2,product_title,VARCHAR,YES,NaN,NaN,NaN
3,category_id,BIGINT,YES,NaN,NaN,NaN
4,prov_id_source_2,INTEGER,YES,NaN,NaN,NaN


In [124]:
con.execute("""DESCRIBE _freamon_source_3_with_prov_view""").df()

,column_name,column_type,null,key,default,extra
0,id,BIGINT,YES,NaN,NaN,NaN
1,category,VARCHAR,YES,NaN,NaN,NaN
2,prov_id_source_3,INTEGER,YES,NaN,NaN,NaN


In [125]:
con.execute("""DESCRIBE _freamon_source_0_with_prov_view""").df()

,column_name,column_type,null,key,default,extra
0,marketplace,VARCHAR,YES,NaN,NaN,NaN
1,customer_id,BIGINT,YES,NaN,NaN,NaN
2,review_id,VARCHAR,YES,NaN,NaN,NaN
3,product_id,VARCHAR,YES,NaN,NaN,NaN
4,vine,VARCHAR,YES,NaN,NaN,NaN
5,third_party,VARCHAR,YES,NaN,NaN,NaN
6,review_headline,VARCHAR,YES,NaN,NaN,NaN
7,review_body,VARCHAR,YES,NaN,NaN,NaN
8,review_date,VARCHAR,YES,NaN,NaN,NaN
9,prov_id_source_0,INTEGER,YES,NaN,NaN,NaN


In [127]:
con.execute("""
SELECT fs0.marketplace, fs3.category, ftv.y_true, ftv.y_pred
FROM _freamon_test_view ftv
JOIN _freamon_source_0_with_prov_view fs0 ON fs0.prov_id_source_0 = ftv.prov_id_source_0
JOIN _freamon_source_3_with_prov_view fs3 ON fs3.prov_id_source_3 = ftv.prov_id_source_3
""")
con.df()

,marketplace,category,y_true,y_pred
0,US,Digital_Video_Games,0,0
1,US,Digital_Video_Games,1,1
2,US,Digital_Video_Games,1,1
3,US,Digital_Video_Games,1,1
4,US,Digital_Video_Games,1,1
...,...,...,...,...
25950,US,Digital_Video_Games,1,1
25951,US,Digital_Video_Games,1,1
25952,US,Digital_Video_Games,0,0
25953,US,Digital_Video_Games,1,1


In [ ]:
pipeline.test_sources

In [117]:
rectangle.columns

Index(['features', 'y_true', 'y_pred', 'prov_id_source_0', 'prov_id_source_1',
       'prov_id_source_2', 'prov_id_source_3', 'marketplace', 'customer_id',
       'review_id', 'product_id', 'vine', 'third_party', 'review_headline',
       'review_body', 'review_date', 'prov_id_source_0_2', 'review_id_2',
       'star_rating', 'helpful_votes', 'total_votes', 'prov_id_source_1_2',
       'product_id_2', 'product_parent', 'product_title', 'category_id',
       'prov_id_source_2_2', 'id', 'category', 'prov_id_source_3_2'],
      dtype='object')